In [39]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import seaborn as sns
import re
import random as rd
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from nltk import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords, wordnet
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import autokeras as ak
from sklearn.model_selection import KFold

import nltk
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ddayv\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ddayv\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [40]:
def Tokenize(f):     ## Pre-processando a frase

    ## Colocando em minusculo
    ## Retirando a pontuaçao
    ## Retirando as StopWords
    
    f = f.lower().replace('\n', '').replace('-','').replace('#','').replace('.','').replace(',','').replace('!','').replace('r\n','').replace('  ','').replace('https','').replace('rt','').replace('rn','')
    token = RegexpTokenizer(r"\w+")
    f = token.tokenize(f)
    
    stop_words = set(stopwords.words('portuguese'))
    
    new_word = [word for word in f if not word in stop_words]
    
    return ' '.join(new_word)

def remove_user(frase):
    frase = re.sub('@\w+','',frase)
    frase = re.sub('{https}[^ ]+','',frase)
    frase = re.sub('https\w+','',frase)
    # re.sub('#\w+','',frase)
    return frase

def pre_X(frases):
    lista = []
    
    for frase in frases:
        lista.append(frase)
        
    return lista

def pre_Y(number):
    lista = []
    
    for numb in number:
        lista.append(numb)
    
    return lista

def set_array(frases):
    
    vocab = []
    palavras = []
    for frase in frases:
        
        text_array = remove_user(frase)
        text_array = Tokenize(text_array)
        text_array = text_array.split(' ')
        for i in range(len(text_array)):
            vocab.append(text_array[i])
    
    
        
    return vocab


## TWEETSENT

In [49]:
df_tweet= pd.read_csv('export_TweetSentBR.csv')
df_tweet

,id,id_twitter,text,sentiment
0,1343,863044774588272640,Que coisa linda! O Programa #encontro estava m...,1
1,1344,865583716088766467,"Por mais #Encontro com as Irmãs Galvão, adorei...",1
2,1345,865063232201011201,Mr. CATRA @OficialMrCatra lançando sua nova mú...,1
3,1346,864668391008763905,quem viu aquela lutadora modela barbuda tatuad...,0
4,1347,865572794016378882,Tô passada com esse cara.... quanta merda pode...,-1
...,...,...,...,...
11565,12908,864636619000877056,eu ja to aqui pronto pro #MasterChefBR mas ain...,-1
11566,12909,863581588713603072,MALUCO! Uma coisa que eu não tenho coragem é e...,-1
11567,12910,864831041349054464,#MaisVoce @ANAMARIABRAGA está linda @RedeGlobo,1
11568,12911,863042798575951872,"Que orgulho de ti, @sportrecife! #Encontro",1


In [50]:
df_remove = df_tweet[df_tweet['sentiment'] == '-']
df_tweet = df_tweet.drop(df_remove.index)

df_tweet['text'] = df_tweet['text'].apply(remove_user)
df_tweet['text'] = df_tweet['text'].apply(Tokenize)

In [51]:
df_tweet['sentiment'] = df_tweet['sentiment'].apply(lambda x: int(x))

In [52]:
text_Tweetsent = df_tweet['text']
polarity = np.asarray(df_tweet['sentiment'])

In [53]:
X_text = count_vect.transform(text_Tweetsent)

X_text_transform = tfidf_transformer.transform(X_text) # Aplicando o TF-IDF
X_text_transform.shape

(11533, 25852)

In [54]:
X_train1, X_test_TWEET, Y_train1, Y_test_TWEET = train_test_split(X_text_transform, polarity, test_size=0.3)

In [55]:
X_train12, X_test_TWEET1, Y_train12, Y_test_TWEET1 = train_test_split(df_tweet['text'], df_tweet['sentiment'] , test_size=0.3)

## TASH

In [56]:
df = pd.read_csv("tash-pt.csv")
df = df.dropna()
df = df.drop(columns=['id_twitter'])


In [57]:
df['text'] = df['text'].apply(remove_user)
df['text'] = df['text'].apply(Tokenize)

In [58]:
text_TASH = df['text']
sentiment = np.asarray(df['sentiment'])

X_text_TASH = count_vect.transform(text_TASH)
X_text_TASH_ = tfidf_transformer.transform(X_text_TASH) # Aplicando o TF-IDF
X_text_TASH_.shape

(2787, 25852)

In [59]:
X_train2, X_test_TASH, Y_train2, Y_test_TASH = train_test_split(X_text_TASH_, sentiment, test_size=0.3)

In [60]:
X_train21, X_test_TASH21, Y_train21, Y_test_TASH21 = train_test_split(df['text'], np.asarray(df['sentiment']), test_size=0.3)

## KANSOAN

In [61]:
df = pd.read_csv("data.txt", sep=',', header=None, names=['text','sentiment'])

df_remove = df[df['sentiment'] == '#Inveja'].index
df = df.drop(df_remove)

df_remove = df[df['sentiment'] == '#Raiva'].index
df = df.drop(df_remove)

df_remove = df[df['sentiment'] == '#Ironia'].index
df = df.drop(df_remove)


def binario(termo):
    if termo == '#Feliz' or termo == '#Amor':
        return 1
    elif termo == '#Triste' or termo == '#Chateado':
        return 0       
    
df['sentiment'] = df['sentiment'].apply(binario)

In [62]:
df['text'] = df['text'].apply(remove_user)
df['text'] = df['text'].apply(Tokenize)

In [63]:
text_KANSOAN = df['text']
sentiment = np.asarray(df['sentiment'])

X_text_KANSOAN = count_vect.transform(text_KANSOAN)
X_text_KANSOAN_ = tfidf_transformer.transform(X_text_KANSOAN) # Aplicando o TF-IDF
X_text_KANSOAN_.shape

(9224, 25852)

In [64]:
X_train3, X_test_KANSOAN, Y_train3, Y_test_KANSOAN = train_test_split(X_text_KANSOAN_, sentiment, test_size=0.3)

In [65]:
X_train31, X_test_KANSOAN1, Y_train31, Y_test_KANSOAN1 = train_test_split(df['text'], np.asarray(df['sentiment']), test_size=0.3)

## TEST

In [41]:
df = pd.read_excel('./rotulação/TweetsPolitical01 OK OK.xlsx')
df2 = pd.read_excel('./rotulação/TweetsPolitical02 OK OK.xlsx')
df3 = pd.read_excel('./rotulação/TweetsPolitical03 OK OK.xlsx')
df4 = pd.read_excel('./rotulação/TweetsPolitical04 OK OK.xlsx')
df5 = pd.read_excel('./rotulação/TweetsPolitical05 OK OK.xlsx')
df6 = pd.read_excel('./rotulação/TweetsPolitical06 OK OK.xlsx')
df7 = pd.read_excel('./rotulação/TweetsPolitical07 OK OK.xlsx')
df8 = pd.read_excel('./rotulação/TweetsPolitical08 OK OK.xlsx')
df9 = pd.read_excel('./rotulação/TweetsPolitical09 OK OK.xlsx')
df10 = pd.read_excel('./rotulação/TweetsPolitical10 OK OK.xlsx')
df11 = pd.read_excel('./rotulação/TweetsPolitical11 OK OK.xlsx')
df12 = pd.read_excel('./rotulação/TweetsPolitical12 OK OK.xlsx')
df13 = pd.read_excel('./rotulação/TweetsPolitical13 OK OK.xlsx')
df14 = pd.read_excel('./rotulação/TweetsPolitical14 OK OK.xlsx')
df15 = pd.read_excel('./rotulação/TweetsPolitical15 OK OK.xlsx')

lista = [df,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15]

df_all = pd.concat(lista)
df_all.shape

(12670, 2)

In [42]:
df_all = df_all.dropna()

In [43]:
df_all.drop(df_all[df_all['Polaridade'] == 11].index, inplace=True)
df_all.drop(df_all[df_all['Polaridade'] == -2].index, inplace=True)
df_all.drop(df_all[df_all['Polaridade'] == 10].index, inplace=True)

df_all['Polaridade'].unique()

array([ 0.,  1., -1.])

In [44]:
df_all.head()

,Tweet,Polaridade
0,- #CaoNossoDeCadaDia #Novo - Vanessa Mandotti ...,0.0
1,"- Bola pra frente que amanhã é outro dia, outr...",1.0
2,- Cara de mal? Acho que não... Apenas um corte...,1.0
3,""" #PCdoB O PARTIDO QUE FAZ A DIFERENÇA: LEAL E...",1.0
4,"""@camaradamae: #PCdoB o Partido da coragem ! ...",1.0


In [45]:
df_all['Tweet'] = df_all['Tweet'].apply(remove_user)
df_all['Tweet'] = df_all['Tweet'].apply(Tokenize)
polarity = np.asarray(df_all['Polaridade'])
Tweet = df_all['Tweet']


In [46]:
count_vect = CountVectorizer()

Tweet = count_vect.fit_transform(Tweet)

tfidf_transformer = TfidfTransformer()
X_train_transform = tfidf_transformer.fit_transform(Tweet) # Aplicando o TF-IDF
X_train_transform.shape

(12626, 25852)

In [47]:
X_train, X_test, Y_train, Y_test = train_test_split(X_train_transform, polarity, test_size=0.3)



In [48]:
X_train_text, X_test_text, Y_train_text, Y_test_text = train_test_split(df_all['Tweet'] , df_all['Polaridade'], test_size=0.3)


## NB - TRAIN -> KANSOAN - TEST -> TWEETSENT

In [66]:
clf = MultinomialNB()
clf.fit(X_train3, Y_train3)
y_pred = clf.predict(X_test_TWEET)

print(classification_report(Y_test_TWEET,y_pred))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00      1013
           0       0.27      0.60      0.38       904
           1       0.57      0.55      0.56      1543

    accuracy                           0.40      3460
   macro avg       0.28      0.38      0.31      3460
weighted avg       0.33      0.40      0.35      3460



c:\users\ddayv\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## NB - TRAIN -> KANSOAN - TEST -> TASH

In [67]:
clf = MultinomialNB()
clf.fit(X_train3, Y_train3)
y_pred = clf.predict(X_test_TASH)

print(classification_report(Y_test_TASH,y_pred))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       281
           0       0.34      0.69      0.46       281
           1       0.47      0.45      0.46       275

    accuracy                           0.38       837
   macro avg       0.27      0.38      0.31       837
weighted avg       0.27      0.38      0.30       837



c:\users\ddayv\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## NB - TRAIN -> KANSOAN - TEST -> UNILEX

In [68]:
def func(num):
    if num == 0:
        return -1
    else:
        return 1

#sentiment = df['sentiment'].apply(func)
#X_train3, X_test_KANSOAN, Y_train3, Y_test_KANSOAN = train_test_split(X_text_KANSOAN_, sentiment, test_size=0.3)

In [69]:
clf = MultinomialNB()
clf.fit(X_train3, Y_train3)
y_pred = clf.predict(X_test)

print(classification_report(Y_test,y_pred, zero_division=True))

              precision    recall  f1-score   support

        -1.0       1.00      0.00      0.00      1286
         0.0       0.34      0.51      0.41      1441
         1.0       0.33      0.52      0.40      1061

    accuracy                           0.34      3788
   macro avg       0.56      0.34      0.27      3788
weighted avg       0.56      0.34      0.27      3788



## SVM - TRAIN->UNILEX - TEST->TWEETSENT

In [70]:
clf = SVC().fit(X_train3, Y_train3) 
y_pred = clf.predict(X_test_TWEET)

print(classification_report(Y_test_TWEET,y_pred))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00      1013
           0       0.27      0.58      0.37       904
           1       0.55      0.55      0.55      1543

    accuracy                           0.40      3460
   macro avg       0.28      0.38      0.31      3460
weighted avg       0.32      0.40      0.34      3460



c:\users\ddayv\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## SVM T U T TASH

In [71]:
clf = SVC().fit(X_train3, Y_train3) 
y_pred = clf.predict(X_test_TASH)

print(classification_report(Y_test_TASH,y_pred))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       281
           0       0.36      0.63      0.46       281
           1       0.41      0.50      0.45       275

    accuracy                           0.38       837
   macro avg       0.25      0.38      0.30       837
weighted avg       0.25      0.38      0.30       837



## SVM T KANSOAN T unilex


In [72]:
clf = SVC().fit(X_train3, Y_train3) 
y_pred = clf.predict(X_test)

print(classification_report(Y_test,y_pred, zero_division=True))

              precision    recall  f1-score   support

        -1.0       1.00      0.00      0.00      1286
         0.0       0.34      0.45      0.39      1441
         1.0       0.31      0.55      0.39      1061

    accuracy                           0.32      3788
   macro avg       0.55      0.33      0.26      3788
weighted avg       0.55      0.32      0.26      3788



## RL T U T TWEETSENT

In [73]:
clf = LogisticRegression(max_iter=1000).fit(X_train3, Y_train3) 
y_pred = clf.predict(X_test_TWEET)

print(classification_report(Y_test_TWEET,y_pred))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00      1013
           0       0.28      0.55      0.37       904
           1       0.55      0.60      0.58      1543

    accuracy                           0.41      3460
   macro avg       0.28      0.38      0.32      3460
weighted avg       0.32      0.41      0.35      3460



c:\users\ddayv\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## RL T U T TASH

In [74]:
clf = LogisticRegression(max_iter=1000).fit(X_train3, Y_train3) 
y_pred = clf.predict(X_test_TASH)

print(classification_report(Y_test_TASH,y_pred))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       281
           0       0.34      0.57      0.43       281
           1       0.40      0.53      0.46       275

    accuracy                           0.37       837
   macro avg       0.25      0.37      0.29       837
weighted avg       0.25      0.37      0.29       837



c:\users\ddayv\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## RL T U T KANSOAN

In [75]:
clf = LogisticRegression(max_iter=1000).fit(X_train3, Y_train3) 
y_pred = clf.predict(X_test)

print(classification_report(Y_test,y_pred, zero_division=True))

              precision    recall  f1-score   support

        -1.0       1.00      0.00      0.00      1286
         0.0       0.31      0.31      0.31      1441
         1.0       0.30      0.66      0.41      1061

    accuracy                           0.30      3788
   macro avg       0.54      0.32      0.24      3788
weighted avg       0.54      0.30      0.23      3788



## TOTALMENT.C T U T TWEETSENT

In [76]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(25, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(10, activation='tanh'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1 , activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss="binary_crossentropy",
    metrics=['accuracy']
)

one = OneHotEncoder(sparse=False)

y_one = np.asarray(Y_train3)
y_one_test = Y_test_TWEET

fit = model.fit(X_train3.todense(), y_one, epochs=10, validation_data=(X_test_TWEET.todense(), y_one_test))

Epoch 1/10
202/202 [==============================] - 2s 12ms/step - loss: 0.2955 - accuracy: 0.9122 - val_loss: 0.3816 - val_accuracy: 0.3991
Epoch 2/10
202/202 [==============================] - 2s 9ms/step - loss: 0.0590 - accuracy: 0.9799 - val_loss: 0.3096 - val_accuracy: 0.3749
Epoch 3/10
202/202 [==============================] - 2s 9ms/step - loss: 0.0256 - accuracy: 0.9932 - val_loss: 0.8454 - val_accuracy: 0.4231
Epoch 4/10
202/202 [==============================] - 2s 9ms/step - loss: 0.0190 - accuracy: 0.9943 - val_loss: 0.6188 - val_accuracy: 0.3887
Epoch 5/10
202/202 [==============================] - 2s 9ms/step - loss: 0.0150 - accuracy: 0.9960 - val_loss: 0.6740 - val_accuracy: 0.3887
Epoch 6/10
202/202 [==============================] - 2s 8ms/step - loss: 0.0118 - accuracy: 0.9957 - val_loss: 0.9175 - val_accuracy: 0.4020
Epoch 7/10
202/202 [==============================] - 2s 8ms/step - loss: 0.0098 - accuracy: 0.9972 - val_loss: 1.3841 - val_accuracy: 0.4263
Epoch

In [77]:
predicted = model.predict(X_test_TWEET.todense())
for i in range(len(predicted)):
    if predicted[i,0] >= 0.5:
        predicted[i,0] = 1
    else:
        predicted[i,0] = -1
        
print(classification_report(y_one_test, predicted))

              precision    recall  f1-score   support

          -1       0.39      0.61      0.48      1013
           0       0.00      0.00      0.00       904
           1       0.54      0.66      0.59      1543

    accuracy                           0.47      3460
   macro avg       0.31      0.42      0.36      3460
weighted avg       0.36      0.47      0.40      3460



c:\users\ddayv\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## LSTM T U T TWEETSENT

In [78]:
vectorize_layer = TextVectorization(
 max_tokens=15000,
 output_mode='int',
 output_sequence_length=len(max(df['text'])))

vocab = set_array(df['text'])

vectorize_layer.adapt(np.unique(vocab))
len(vectorize_layer.get_vocabulary())


15000

In [79]:
text_KANSOAN

X_train3_, X_test_KANSOAN_, Y_train3_, Y_test_KANSOAN_ = train_test_split(text_KANSOAN, df['sentiment'], test_size=0.3)

In [80]:
model = tf.keras.Sequential([
    vectorize_layer,
    tf.keras.layers.Embedding(
        input_dim=len(vectorize_layer.get_vocabulary()),
        output_dim=64,mask_zero=True),
    
    tf.keras.layers.LSTM(50, activation='relu' ,return_sequences=True),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.LSTM(25 , activation='tanh', return_sequences=True),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.LSTM(10 , activation='tanh'),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Dense(1, activation='sigmoid')
])


model.compile(
    optimizer= tf.keras.optimizers.Adam(),
    loss="binary_crossentropy",
    metrics=['accuracy']
)

y_one = np.asarray(Y_train3_)
y_one_test = Y_test_TWEET1

fit = model.fit(np.asarray(pre_X(X_train3_)), y_one, epochs=10, batch_size=128 ,validation_data=(np.asarray(pre_X(X_test_TWEET1)), y_one_test))

Epoch 1/10
51/51 [==============================] - 9s 173ms/step - loss: 0.6604 - accuracy: 0.5686 - val_loss: 0.9342 - val_accuracy: 0.4344
Epoch 2/10
51/51 [==============================] - 7s 133ms/step - loss: 0.3559 - accuracy: 0.8933 - val_loss: 0.9871 - val_accuracy: 0.4150
Epoch 3/10
51/51 [==============================] - 7s 133ms/step - loss: 0.1516 - accuracy: 0.9669 - val_loss: 0.9188 - val_accuracy: 0.4118
Epoch 4/10
51/51 [==============================] - 7s 133ms/step - loss: 0.0744 - accuracy: 0.9865 - val_loss: 0.6938 - val_accuracy: 0.3919
Epoch 5/10
51/51 [==============================] - 7s 132ms/step - loss: 0.0489 - accuracy: 0.9915 - val_loss: 1.1733 - val_accuracy: 0.4098
Epoch 6/10
51/51 [==============================] - 7s 134ms/step - loss: 0.0334 - accuracy: 0.9943 - val_loss: 1.4608 - val_accuracy: 0.4214
Epoch 7/10
51/51 [==============================] - 7s 131ms/step - loss: 0.0275 - accuracy: 0.9949 - val_loss: 1.0310 - val_accuracy: 0.4009
Epoch 

In [81]:
predicted = model.predict(np.asarray(pre_X(X_test_TWEET1)))

for i in range(len(predicted)):
    if predicted[i,0] >= 0.5:
        predicted[i,0] = 1
    else:
        predicted[i,0] = -1

print(classification_report(y_one_test, predicted))

              precision    recall  f1-score   support

          -1       0.38      0.60      0.46      1006
           0       0.00      0.00      0.00       896
           1       0.53      0.63      0.57      1558

    accuracy                           0.46      3460
   macro avg       0.30      0.41      0.35      3460
weighted avg       0.35      0.46      0.39      3460



## UNILEX

In [82]:
predicted = model.predict(np.asarray(pre_X(X_test_text)))

for i in range(len(predicted)):
    if predicted[i,0] >= 0.5:
        predicted[i,0] = 1
    else:
        predicted[i,0] = -1

print(classification_report(np.asarray(Y_test_text), predicted))

              precision    recall  f1-score   support

        -1.0       0.46      0.42      0.44      1302
         0.0       0.00      0.00      0.00      1392
         1.0       0.31      0.73      0.43      1094

    accuracy                           0.35      3788
   macro avg       0.25      0.38      0.29      3788
weighted avg       0.25      0.35      0.28      3788



## TASH

In [83]:
predicted = model.predict(np.asarray(pre_X(X_test_TASH21)))

for i in range(len(predicted)):
    if predicted[i,0] >= 0.5:
        predicted[i,0] = 1
    else:
        predicted[i,0] = -1

print(classification_report(np.asarray(Y_test_TASH21), predicted))

              precision    recall  f1-score   support

          -1       0.36      0.44      0.40       256
           0       0.00      0.00      0.00       290
           1       0.37      0.66      0.47       291

    accuracy                           0.37       837
   macro avg       0.24      0.37      0.29       837
weighted avg       0.24      0.37      0.29       837



## CONV1D T U T TWEET

In [84]:
model = tf.keras.Sequential([
    vectorize_layer,
    tf.keras.layers.Embedding(
        input_dim=len(vectorize_layer.get_vocabulary()),
        output_dim=64,
        mask_zero=True),
    
    tf.keras.layers.Conv1D(32,6, activation='relu'),
    tf.keras.layers.MaxPooling1D(2),
    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')   
])

model.compile(
    optimizer= tf.keras.optimizers.Adam(),
    loss="binary_crossentropy",
    metrics=['accuracy']
)

y_one = Y_train3_
y_one_test = Y_test_TWEET1

fit = model.fit(np.asarray(pre_X(X_train3_)), y_one, epochs=10, batch_size=128 ,validation_data=(np.asarray(pre_X(X_test_TWEET1)), y_one_test))

Epoch 1/10
51/51 [==============================] - 2s 30ms/step - loss: 0.6281 - accuracy: 0.6798 - val_loss: 0.9824 - val_accuracy: 0.3942
Epoch 2/10
51/51 [==============================] - 2s 30ms/step - loss: 0.1514 - accuracy: 0.9535 - val_loss: 1.1852 - val_accuracy: 0.4419
Epoch 3/10
51/51 [==============================] - 1s 25ms/step - loss: 0.0343 - accuracy: 0.9890 - val_loss: 1.4137 - val_accuracy: 0.4358
Epoch 4/10
51/51 [==============================] - 1s 27ms/step - loss: 0.0122 - accuracy: 0.9971 - val_loss: 1.1418 - val_accuracy: 0.4237
Epoch 5/10
51/51 [==============================] - 1s 28ms/step - loss: 0.0076 - accuracy: 0.9978 - val_loss: 1.3557 - val_accuracy: 0.4249
Epoch 6/10
51/51 [==============================] - 1s 25ms/step - loss: 0.0045 - accuracy: 0.9988 - val_loss: 1.6143 - val_accuracy: 0.4306
Epoch 7/10
51/51 [==============================] - 1s 28ms/step - loss: 0.0036 - accuracy: 0.9992 - val_loss: 1.4842 - val_accuracy: 0.4283
Epoch 8/10
51

In [85]:
predicted = model.predict(np.asarray(pre_X(X_test_TWEET1)))

for i in range(len(predicted)):
    if predicted[i,0] >= 0.5:
        predicted[i,0] = 1
    else:
        predicted[i,0] = -1

print(classification_report(y_one_test, predicted))

              precision    recall  f1-score   support

          -1       0.41      0.49      0.45      1006
           0       0.00      0.00      0.00       896
           1       0.52      0.76      0.61      1558

    accuracy                           0.48      3460
   macro avg       0.31      0.41      0.35      3460
weighted avg       0.35      0.48      0.41      3460



## UNILEX

In [86]:
predicted = model.predict(np.asarray(pre_X(X_test_text)))

for i in range(len(predicted)):
    if predicted[i,0] >= 0.5:
        predicted[i,0] = 1
    else:
        predicted[i,0] = -1

print(classification_report(np.asarray(Y_test_text), predicted))

              precision    recall  f1-score   support

        -1.0       0.46      0.24      0.32      1302
         0.0       0.00      0.00      0.00      1392
         1.0       0.29      0.83      0.43      1094

    accuracy                           0.32      3788
   macro avg       0.25      0.36      0.25      3788
weighted avg       0.24      0.32      0.23      3788



## TASH

In [87]:
predicted = model.predict(np.asarray(pre_X(X_test_TASH21)))

for i in range(len(predicted)):
    if predicted[i,0] >= 0.5:
        predicted[i,0] = 1
    else:
        predicted[i,0] = -1

print(classification_report(np.asarray(Y_test_TASH21), predicted))

              precision    recall  f1-score   support

          -1       0.37      0.30      0.33       256
           0       0.00      0.00      0.00       290
           1       0.36      0.77      0.49       291

    accuracy                           0.36       837
   macro avg       0.24      0.36      0.27       837
weighted avg       0.24      0.36      0.27       837



## BDR T U T TWEET

In [88]:
model = tf.keras.Sequential([
    vectorize_layer,
    tf.keras.layers.Embedding(
        input_dim=len(vectorize_layer.get_vocabulary()),
        output_dim=1,
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(1)),

    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer= tf.keras.optimizers.Adam(),
    loss="binary_crossentropy",
    metrics=['accuracy']
)

y_one = Y_train3_
y_one_test = Y_test_TWEET1

fit = model.fit(np.asarray(pre_X(X_train3_)), y_one, epochs=10, batch_size=128 ,validation_data=(np.asarray(pre_X(X_test_TWEET1)), y_one_test))

Epoch 1/10
51/51 [==============================] - 3s 65ms/step - loss: 0.6922 - accuracy: 0.5308 - val_loss: 0.7097 - val_accuracy: 0.4503
Epoch 2/10
51/51 [==============================] - 1s 25ms/step - loss: 0.6889 - accuracy: 0.5178 - val_loss: 0.7269 - val_accuracy: 0.4503
Epoch 3/10
51/51 [==============================] - 1s 27ms/step - loss: 0.6810 - accuracy: 0.5178 - val_loss: 0.7472 - val_accuracy: 0.4503
Epoch 4/10
51/51 [==============================] - 1s 26ms/step - loss: 0.6617 - accuracy: 0.5356 - val_loss: 0.7713 - val_accuracy: 0.4503
Epoch 5/10
51/51 [==============================] - 1s 25ms/step - loss: 0.6221 - accuracy: 0.7297 - val_loss: 0.7892 - val_accuracy: 0.4480
Epoch 6/10
51/51 [==============================] - 1s 25ms/step - loss: 0.5556 - accuracy: 0.9023 - val_loss: 0.8246 - val_accuracy: 0.4436
Epoch 7/10
51/51 [==============================] - 1s 26ms/step - loss: 0.4781 - accuracy: 0.9181 - val_loss: 0.8500 - val_accuracy: 0.4393
Epoch 8/10
51

In [89]:
predicted = model.predict(np.asarray(pre_X(X_test_TWEET1)))

for i in range(len(predicted)):
    if predicted[i,0] >= 0.5:
        predicted[i,0] = 1
    else:
        predicted[i,0] = -1

print(classification_report(y_one_test, predicted))

              precision    recall  f1-score   support

          -1       0.42      0.40      0.41      1006
           0       0.00      0.00      0.00       896
           1       0.50      0.80      0.62      1558

    accuracy                           0.48      3460
   macro avg       0.31      0.40      0.34      3460
weighted avg       0.35      0.48      0.40      3460



## UNILEX

In [90]:
predicted = model.predict(np.asarray(pre_X(X_test_text)))

for i in range(len(predicted)):
    if predicted[i,0] >= 0.5:
        predicted[i,0] = 1
    else:
        predicted[i,0] = -1

print(classification_report(np.asarray(Y_test_text), predicted))

              precision    recall  f1-score   support

        -1.0       0.41      0.15      0.22      1302
         0.0       0.00      0.00      0.00      1392
         1.0       0.29      0.89      0.44      1094

    accuracy                           0.31      3788
   macro avg       0.23      0.34      0.22      3788
weighted avg       0.22      0.31      0.20      3788



## TASH

In [91]:
predicted = model.predict(np.asarray(pre_X(X_test_TASH21)))

for i in range(len(predicted)):
    if predicted[i,0] >= 0.5:
        predicted[i,0] = 1
    else:
        predicted[i,0] = -1

print(classification_report(np.asarray(Y_test_TASH21), predicted))

              precision    recall  f1-score   support

          -1       0.38      0.33      0.35       256
           0       0.00      0.00      0.00       290
           1       0.36      0.77      0.50       291

    accuracy                           0.37       837
   macro avg       0.25      0.37      0.28       837
weighted avg       0.24      0.37      0.28       837

